# Here I will try to train an XGBOOST ML to Predict Fixation Time for Graphs

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split